###  Introduction

All the datasets used in the preceding chapters have not had much or any work done to
change their structure. We immediately began processing the datasets in their original shape.
Many datasets in the wild will need a significant amount of restructuring before commencing
a more detailed analysis. In some cases, an entire project might only concern itself with
formatting the data in such a way that it can be easily processed by someone else.
There are many terms that are used to describe the process of data restructuring, with tidy
data being the most common to data scientists. Tidy data is a term coined by Hadley Wickham
to describe a form of data that makes analysis easy to do. This chapter will cover many
ideas formulated by Hadley and how to accomplish them with pandas. To learn a great deal
more about tidy data, read Hadley's paper (http://vita.had.co.nz/papers/tidydata.
pdf).

What is tidy data? Hadley puts forth three guiding principles that determine whether a dataset
is tidy:

 Each variable forms a column
 
 Each observation forms a row
 
 Each type of observational unit forms a table
 
Any dataset that does not meet these guidelines is considered messy. This definition will
make more sense once we start restructuring our data into tidy form, but for now, we'll need
to know what variables, observations, and observational units are.
Using this jargon, a variable is not referring to a Python variable, it is a piece of data. It is good
to think about the distinction between a variable name and the variable value. The variable
names are labels, such as gender, race, salary, and position. The variable values are those
things liable to change for every observation, such as male, female, or other for gender.
A single observation is the collection of all variable values for a single observational unit.
To help understand what an observational unit might be, consider a retail store, which has
data for each transaction, employee, customer, item, and the store itself. Each of these can
be viewed as an observational unit and would require its own table. Combining employee
information (like the number of hours worked) with customer information (like the amount
spent) in the same table would break this tidy principle.
The first step to resolving messy data is to recognize it when it exists, and there are boundless
possibilities. Hadley explicitly mentions five of the most common types of messy data:

 Column names are values, not variable names
 
 Multiple variables are stored in column names
 
 Variables are stored in both rows and columns
 
 Multiple types of observational units are stored in the same table
 
 A single observational unit is stored in multiple tables

It is important to understand that tidying data does not typically involve changing the values
of your dataset, filling in missing values, or doing any sort of analysis. Tidying data consists
in changing the shape or structure of the data to meet the tidy principles. Tidy data is akin
to having all your tools in the toolbox instead of scattered randomly throughout your house.
Having the tools properly in the toolbox allows all other tasks to be completed easily. Once
the data is in the correct form, it becomes much easier to perform further analysis.

Once you have spotted messy data, you will use the pandas library to restructure the data,
so that it is tidy. The main tidy tools that pandas has available for you are the DataFrame
methods .stack, .melt, .unstack, and .pivot. More complex tidying involves ripping
apart text, which necessitates the .str accessor. Other helper methods, such as .rename,
.rename_axis, .reset_index, and .set_index, will help with applying the final touches
to tidy data.

## Tidying variable values as column names with stack

In [2]:
# To help understand the differences between tidy and
# messy data, let's take a look at a table that may or 
# may not be in tidy form:
import pandas as pd
import numpy as np

pd.set_option("max_columns", 4, 'max_rows', 15)

In [4]:
state_fruit = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/state_fruit.csv', index_col=0)

In [9]:
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [8]:
state_fruit.T

,Texas,Arizona,Florida
Apple,12,9,0
Orange,10,7,14
Banana,40,12,190


In [10]:
# Note that in this case, the result collapses from a 
# DataFrame to a Series:
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

Notice that we now have a Series with a MultiIndex. There are now two levels in
the index. The original index has been pushed to the left to make room for the fruit
column names. With this one command, we now essentially have tidy data. Each
variable, state, fruit, and weight is vertical. Let's use the .reset_index method
to turn the result into a DataFrame:

In [12]:
(state_fruit
 .stack()
 .reset_index()
)

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [13]:
# Our structure is now correct, but the column names 
# are meaningless. Let's replace them with proper
# identifiers:
(state_fruit
 .stack()
 .reset_index()
 .rename(columns={'level_0': 'state',
                  'level_1': 'fruit', 0: 'weight'})
)

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [14]:
# Instead of using the .rename method, it is possible
# to use the lesser-known Series method .rename_axis to
# set the names of the index levels before using
# .reset_index:

(state_fruit
 .stack()
 .rename_axis(['state', 'fruit'])
)

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [15]:
# From here, we can chain the .reset_index method with 
# the name parameter to reproduce the output from step 3:
(state_fruit
 .stack()
 .rename_axis(['state', 'fruit'])
 .reset_index(name='weight')
)


,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


One of the keys to using .stack is to place all of the columns that you do not wish to
transform in the index. The dataset in this recipe was initially read with the states in the index.
Let's take a look at what would have happened if we did not read the states into the index

In [16]:
state_fruit2 = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/state_fruit2.csv')

In [17]:
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [18]:
# As the state names are not in the index, using .stack
# on this DataFrame reshapes all values into one long 
# Series of values:

state_fruit2.stack()

0  State       Texas
   Apple          12
   Orange         10
   Banana         40
1  State     Arizona
   Apple           9
   Orange          7
   Banana         12
2  State     Florida
   Apple           0
   Orange         14
   Banana        190
dtype: object

This command reshapes all the columns, this time including the states, and is not at all what
we need. To reshape this data correctly, you will need to put all the non-reshaped columns
into the index first with the .set_index method, and then use .stack. The following code
gives a similar result to step 1:

In [21]:
(state_fruit2
 .set_index('State')
 .stack()
 
)

State          
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [22]:
(state_fruit2
 .set_index('State')
 .stack()
 .reset_index()
 
)

,State,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [29]:
(state_fruit2
 .set_index('State')
 .stack()
 .reset_index()
 )
 


,State,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [28]:
(state_fruit2
 .set_index('State')
 .stack()
 .reset_index()
 .rename(columns={'level_1':'Fruits', 0:'weight' })
 
)

,State,Fruits,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


### Tidying variables value as column names with melt

Like most large Python libraries, pandas has many different ways to accomplish the same
task, the differences usually being readability and performance. A DataFrame has a method
named .melt that is similar to the .stack method described in the previous recipe but gives
a bit more flexibility.
In this recipe, we use the .melt method to tidy a DataFrame with variable values as column
names.

In [30]:
# Read in the state_fruit2.csv dataset:
state_fruit2 = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/state_fruit2.csv')

In [31]:
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [32]:
# Use the .melt method by passing the appropriate columns
# to the id_vars and value_vars parameters:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


This one step creates tidy data for us. By default, .melt refers to the transformed
column names as variables and the corresponding values as values. Conveniently,
.melt has two additional parameters, var_name and value_name, that give you
the ability to rename these two columns:

In [33]:
state_fruit2.melt(id_vars=['State'], 
                 value_vars=['Apple', 'Orange', 'Banana'],
                 var_name='Fruit',
                 value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [36]:
# All the parameters for the .melt method are optional, 
# and if you desire all your values to be in a single 
# column and their old column labels to be in the other,
# you may call .melt with the default parameters:
state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
5,Apple,0
6,Orange,10
7,Orange,7
8,Orange,14
9,Banana,40


More realistically, you might have lots of variables that need melting and would like to specify
only the identification variables. In that case, calling .melt in the following manner will yield
the same result as in step 2. You don't even need a list when melting a single column and can
pass its string value:

In [38]:
state_fruit2.melt(id_vars=['State'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


### Stacking multiple groups of variables simultaneously

Some datasets contain multiple groups of variables as column names that need to be
stacked simultaneously into their own columns. An example involving the movie dataset can
help clarify this. Let's begin by selecting all columns containing the actor names and their
corresponding Facebook likes:

In [39]:
movie = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/movie.csv')

In [40]:
movie.head()

,color,director_name,...,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,...,1.78,33000
1,Color,Gore Verbinski,...,2.35,0
2,Color,Sam Mendes,...,2.35,85000
3,Color,Christopher Nolan,...,2.35,164000
4,NaN,Doug Walker,...,NaN,0


In [41]:
actor = movie[['movie_title', 'actor_1_name',
              'actor_2_name', 'actor_3_name',
              'actor_1_facebook_likes',
              'actor_2_facebook_likes',
              'actor_3_facebook_likes']]

In [42]:
actor.head()

,movie_title,actor_1_name,...,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,...,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,...,5000.0,1000.0
2,Spectre,Christoph Waltz,...,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,...,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,...,12.0,NaN


If we define our variables as the title of the movie, the actor name, and the number of Facebook likes, then we will need to stack two sets of columns, which is not possible using
a single call to .stack or .melt.
In this recipe, we will tidy our actor DataFrame by simultaneously stacking the actor names
and their corresponding Facebook likes with the wide_to_long function.

We will be using the wide_to_long function to reshape our data into tidy form.
To use this function, we will need to change the column names that we are stacking,
so that they end with a digit. We first create a user-defined function to change the
column names:

In [45]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = (col_name[:5] + col_name[fb_idx - 1] 
                   + col_name[5: fb_idx-1])
    return col_name

In [46]:
# Pass this function to the rename method to transform 
# all the column names
actor2 = actor.rename(columns=change_col_name)


In [47]:
actor2

,movie_title,actor_1,...,actor__2,actor__3
0,Avatar,CCH Pounder,...,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,...,5000.0,1000.0
2,Spectre,Christoph Waltz,...,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,...,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,...,12.0,NaN
...,...,...,...,...,...
4911,Signed Sealed Delivered,Eric Mabius,...,470.0,318.0
4912,The Following,Natalie Zea,...,593.0,319.0
4913,A Plague So Pleasant,Eva Boehnke,...,0.0,0.0
4914,Shanghai Calling,Alan Ruck,...,719.0,489.0


In [49]:
actor2.columns

Index(['movie_title', 'actor_1', 'actor_2', 'actor_3', 'actor__1', 'actor__2',
       'actor__3'],
      dtype='object')

In [50]:
# Use the wide_to_long function to stack the actor 
# and Facebook sets of columns simultaneously
stubs = ['actor', 'actor_facebook_likes']

actor2_tidy = pd.wide_to_long(actor2,
                              stubnames=stubs,
                              i=['movie_title'],
                              j='actor_num',
                              sep='_'
                             )

In [52]:
actor2_tidy.head()

,,actor__2,actor__1,...,actor,actor_facebook_likes
movie_title,actor_num,,,,,
Avatar,1,936.0,1000.0,...,CCH Pounder,NaN
Pirates of the Caribbean: At World's End,1,5000.0,40000.0,...,Johnny Depp,NaN
Spectre,1,393.0,11000.0,...,Christoph Waltz,NaN
The Dark Knight Rises,1,23000.0,27000.0,...,Tom Hardy,NaN
Star Wars: Episode VII - The Force Awakens,1,12.0,131.0,...,Doug Walker,NaN


The function wide_to_long works when all groupings of variables have the same numeric
ending like they did in this recipe. When your variables do not have the same ending or don't
end in a digit, you can still use wide_to_long to do simultaneous column stacking. For
instance, let's take a look at the following dataset:

In [56]:
pd.set_option('max_column', None)
df = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/stackme.csv')

In [57]:
df

,State,Country,a1,b2,Test,d,e
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [58]:
df.rename(columns={'al': 'group_al', 'b2': 'group_b2',
                  'd': 'group2_al', 'e': 'group2_b2'})


,State,Country,a1,group_b2,Test,group2_al,group2_b2
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [60]:
# We would then need to modify the suffix parameter,
# which normally defaults to a regular expression that 
# selects digits. Here, we tell it to find any number
# of characters:
pd.wide_to_long(
    df.rename(columns={'al': 'group_al', 'b2': 'group_b2',
                  'd': 'group2_al', 'e': 'group2_b2'}),
    stubnames=['group1', 'group2'],
   i=['State', 'Country', 'Test'],
    j='Label', 
    suffix='.+',
    sep='_'
    

)

group_b2    a1  group1  group2
State Country Test  Label                                
TX    US      Test1 al          0.3  0.45     NaN       2
                    b2          0.3  0.45     NaN       6
MA    US      Test2 al          1.2  0.03     NaN       9
                    b2          1.2  0.03     NaN       7
ON    CAN     Test3 al          4.2  0.70     NaN       4
                    b2          4.2  0.70     NaN       2

### Inverting stacked data

DataFrames have two similar methods, .stack and .melt, to convert horizontal column names
into vertical column values. DataFrames can invert these two operations with the .unstack and
.pivot methods, respectively. .stack and .unstack are methods that allow control over only the
column and row indexes, while .melt and .pivot give more flexibility to choose which columns
are reshaped. In this recipe, we will call .stack and .melt on a dataset and promptly invert the operation
with the .unstack and .pivot methods.

In [63]:
# Read in the college dataset with the institution name
# as the index, and with only the undergraduate race
# columns:
pd.set_option('max_columns', 5, 'max_rows', 14)
def usecol_func(name):
    return 'UGDS_' in name or name == 'INSTNM'

college = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/college.csv', index_col='INSTNM', usecols=usecol_func)

In [64]:
college

,UGDS_WHITE,UGDS_BLACK,...,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,
Alabama A & M University,0.0333,0.9353,...,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,...,0.0179,0.0100
Amridge University,0.2990,0.4192,...,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,...,0.0332,0.0350
Alabama State University,0.0158,0.9208,...,0.0243,0.0137
...,...,...,...,...,...
SAE Institute of Technology San Francisco,NaN,NaN,...,NaN,NaN
Rasmussen College - Overland Park,NaN,NaN,...,NaN,NaN
National Personal Training Institute of Cleveland,NaN,NaN,...,NaN,NaN


In [65]:
# Use the .stack method to convert each horizontal column
# name to a vertical index level:
college_stacked = college.stack()

In [66]:
college_stacked

INSTNM                                     
Alabama A & M University         UGDS_WHITE    0.0333
                                 UGDS_BLACK    0.9353
                                 UGDS_HISP     0.0055
                                 UGDS_ASIAN    0.0019
                                 UGDS_AIAN     0.0024
                                                ...  
Coastal Pines Technical College  UGDS_AIAN     0.0034
                                 UGDS_NHPI     0.0017
                                 UGDS_2MOR     0.0191
                                 UGDS_NRA      0.0028
                                 UGDS_UNKN     0.0056
Length: 61866, dtype: float64

In [67]:
# Invert this stacked data back to its original form
# with the .unstack method:
college_stacked.unstack()

,UGDS_WHITE,UGDS_BLACK,...,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,
Alabama A & M University,0.0333,0.9353,...,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,...,0.0179,0.0100
Amridge University,0.2990,0.4192,...,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,...,0.0332,0.0350
Alabama State University,0.0158,0.9208,...,0.0243,0.0137
...,...,...,...,...,...
Hollywood Institute of Beauty Careers-West Palm Beach,0.2182,0.4182,...,0.0182,0.0909
Hollywood Institute of Beauty Careers-Casselberry,0.1200,0.3333,...,0.0000,0.0667
Coachella Valley Beauty College-Beaumont,0.3284,0.1045,...,0.0000,0.0000


In [68]:
# A similar sequence of operations can be done with .melt 
# followed by .pivot. First, read in the data without
# putting the institution name in the index:

college2 = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/college.csv', usecols=usecol_func)

In [69]:
college2

,INSTNM,UGDS_WHITE,...,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,...,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,...,0.0179,0.0100
2,Amridge University,0.2990,...,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,...,0.0332,0.0350
4,Alabama State University,0.0158,...,0.0243,0.0137
...,...,...,...,...,...
7530,SAE Institute of Technology San Francisco,NaN,...,NaN,NaN
7531,Rasmussen College - Overland Park,NaN,...,NaN,NaN
7532,National Personal Training Institute of Cleveland,NaN,...,NaN,NaN
7533,Bay Area Medical Academy - San Jose Satellite ...,NaN,...,NaN,NaN


In [74]:
# Use the .melt method to transpose all the race
# columns into a single column:
college_melted = college2.melt(id_vars='INSTNM',
                              var_name='Race',
                               value_name='Percentage'
                              )

In [75]:
college_melted

,INSTNM,Race,Percentage
0,Alabama A & M University,UGDS_WHITE,0.0333
1,University of Alabama at Birmingham,UGDS_WHITE,0.5922
2,Amridge University,UGDS_WHITE,0.2990
3,University of Alabama in Huntsville,UGDS_WHITE,0.6988
4,Alabama State University,UGDS_WHITE,0.0158
...,...,...,...
67810,SAE Institute of Technology San Francisco,UGDS_UNKN,NaN
67811,Rasmussen College - Overland Park,UGDS_UNKN,NaN
67812,National Personal Training Institute of Cleveland,UGDS_UNKN,NaN
67813,Bay Area Medical Academy - San Jose Satellite ...,UGDS_UNKN,NaN


In [76]:
# Use the .pivot method to invert this previous result:
melted_inv = college_melted.pivot(index='INSTNM', columns='Race',
                                  values='Percentage')

melted_inv

Race,UGDS_2MOR,UGDS_AIAN,...,UGDS_UNKN,UGDS_WHITE
INSTNM,,,,,
A & W Healthcare Educators,0.0000,0.0000,...,0.0000,0.0000
A T Still University of Health Sciences,NaN,NaN,...,NaN,NaN
ABC Beauty Academy,0.0000,0.0000,...,0.0000,0.0000
ABC Beauty College Inc,0.0000,0.0000,...,0.0000,0.2895
AI Miami International University of Art and Design,0.0018,0.0000,...,0.4644,0.0324
...,...,...,...,...,...
Yukon Beauty College Inc,0.0000,0.1200,...,0.0000,0.8000
Z Hair Academy,0.0211,0.0000,...,0.0105,0.9368
Zane State College,0.0218,0.0029,...,0.2399,0.6995


Notice that the institution names are now shuttled over into the index and are not in
their original order. The column names are not in their original order. To get an exact
replication of our starting DataFrame from step 4, use the .loc index operator to
select rows and columns simultaneously and then reset the index:

In [77]:
college2_replication = (melted_inv
                       .loc[college2['INSTNM'], college2.columns[1:]]
                       .reset_index())

In [78]:
college2_replication

,INSTNM,UGDS_WHITE,...,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,...,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,...,0.0179,0.0100
2,Amridge University,0.2990,...,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,...,0.0332,0.0350
4,Alabama State University,0.0158,...,0.0243,0.0137
...,...,...,...,...,...
7530,SAE Institute of Technology San Francisco,NaN,...,NaN,NaN
7531,Rasmussen College - Overland Park,NaN,...,NaN,NaN
7532,National Personal Training Institute of Cleveland,NaN,...,NaN,NaN
7533,Bay Area Medical Academy - San Jose Satellite ...,NaN,...,NaN,NaN


In [79]:
college2.equals(college2_replication)

True

If you take a look at the output from step 2, you'll notice that there are two index levels. By
default, the .unstack method uses the innermost index level as the new column values.
Index levels are numbered beginning from zero from the outside. pandas defaults the level
parameter of the .unstack method to -1, which refers to the innermost index. We can
instead .unstack the outermost column using level=0:

In [80]:
college.stack().unstack(0)

INSTNM,Alabama A & M University,University of Alabama at Birmingham,...,Dewey University-Mayaguez,Coastal Pines Technical College
UGDS_WHITE,0.0333,0.5922,...,0.0,0.6762
UGDS_BLACK,0.9353,0.2600,...,0.0,0.2508
UGDS_HISP,0.0055,0.0283,...,1.0,0.0359
UGDS_ASIAN,0.0019,0.0518,...,0.0,0.0045
UGDS_AIAN,0.0024,0.0022,...,0.0,0.0034
UGDS_NHPI,0.0019,0.0007,...,0.0,0.0017
UGDS_2MOR,0.0000,0.0368,...,0.0,0.0191
UGDS_NRA,0.0059,0.0179,...,0.0,0.0028
UGDS_UNKN,0.0138,0.0100,...,0.0,0.0056


In [81]:
# There is a way to transpose a DataFrame that does not
# require .stack or .unstack. Use the .transpose method 
# or the .T attribute like this:

college.T

INSTNM,Alabama A & M University,University of Alabama at Birmingham,...,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,...,NaN,NaN
UGDS_BLACK,0.9353,0.2600,...,NaN,NaN
UGDS_HISP,0.0055,0.0283,...,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,...,NaN,NaN
UGDS_AIAN,0.0024,0.0022,...,NaN,NaN
UGDS_NHPI,0.0019,0.0007,...,NaN,NaN
UGDS_2MOR,0.0000,0.0368,...,NaN,NaN
UGDS_NRA,0.0059,0.0179,...,NaN,NaN
UGDS_UNKN,0.0138,0.0100,...,NaN,NaN


## Unstacking after a groupby aggregation

Grouping data by a single column and performing an aggregation on a single column returns
a result that is easy to consume. When grouping by more than one column, a resulting
aggregation might not be structured in a manner that makes consumption easy. Since
.groupby operations, by default, put the unique grouping columns in the index, the .unstack
method can be beneficial to rearrange the data so that it is presented in a manner that is
more useful for interpretation.
In this recipe, we use the employee dataset to perform an aggregation, grouping by multiple
columns. We then use the .unstack method to reshape the result into a format that makes
for easier comparisons of different groups.

In [82]:
employee = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/employee.csv')

In [83]:
(employee
 .groupby('RACE')
 ['BASE_SALARY']
 .mean()
 .astype(int)
)

RACE
American Indian or Alaskan Native    60272
Asian/Pacific Islander               61660
Black or African American            50137
Hispanic/Latino                      52345
Others                               51278
White                                64419
Name: BASE_SALARY, dtype: int32

In [84]:
# This is a groupby operation that results in a Series 
# that is easy to read and has no need to reshape.
# Let's now find the average salary for all races by
# gender. Note that the result is a Series:

(employee
 .groupby(['RACE', 'GENDER'])
 ['BASE_SALARY']
 .mean()
 .astype(int)
)

RACE                               GENDER
American Indian or Alaskan Native  Female    60238
                                   Male      60305
Asian/Pacific Islander             Female    63226
                                   Male      61033
Black or African American          Female    48915
                                   Male      51082
Hispanic/Latino                    Female    46503
                                   Male      54782
Others                             Female    63785
                                   Male      38771
White                              Female    66793
                                   Male      63940
Name: BASE_SALARY, dtype: int32

This aggregation is more complex and can be reshaped to make different
comparisons easier. For instance, it would be easier to compare male versus female
salaries for each race if they were side by side and not vertical as they are now. Let's
call on .unstack on the gender index level:

In [85]:
(employee
 .groupby(['RACE', 'GENDER'])
 ['BASE_SALARY']
 .mean()
 .astype(int)
 .unstack('GENDER')
)

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238,60305
Asian/Pacific Islander,63226,61033
Black or African American,48915,51082
Hispanic/Latino,46503,54782
Others,63785,38771
White,66793,63940


In [86]:
# Similarly, we can unstack the race index level:
(employee
 .groupby(['RACE', 'GENDER'])
 ['BASE_SALARY']
 .mean()
 .astype(int)
 .unstack('RACE')
)

RACE,American Indian or Alaskan Native,Asian/Pacific Islander,...,Others,White
GENDER,,,,,
Female,60238,63226,...,63785,66793
Male,60305,61033,...,38771,63940


If there are multiple aggregating functions when performing a groupby with a single column
from a DataFrame, then the immediate result will be a DataFrame and not a Series. For
instance, let's calculate more aggregations than just the mean, as was done in step 2:

In [88]:
(employee
 .groupby(['RACE', 'GENDER'])
 ['BASE_SALARY']
 .agg(['mean', 'min', 'max'])
 .astype(int)
 
)

mean    min     max
RACE                              GENDER                      
American Indian or Alaskan Native Female  60238  26125   98536
                                  Male    60305  26125   81239
Asian/Pacific Islander            Female  63226  26125  130416
                                  Male    61033  27914  163228
Black or African American         Female  48915  24960  150416
                                  Male    51082  26125  275000
Hispanic/Latino                   Female  46503  26125  126115
                                  Male    54782  26104  165216
Others                            Female  63785  63785   63785
                                  Male    38771  38771   38771
White                             Female  66793  27955  178331
                                  Male    63940  26125  210588

Unstacking the Gender column will result in columns with a MultiIndex. From here, you
can keep swapping row and column levels with both the .unstack and .stack methods
until you achieve the structure of data you desire

In [89]:
(employee
 .groupby(['RACE', 'GENDER'])
 ['BASE_SALARY']
 .agg(['mean', 'min', 'max'])
 .astype(int)
 .unstack('GENDER')
 
)

mean         ...     max        
GENDER                            Female   Male  ...  Female    Male
RACE                                             ...                
American Indian or Alaskan Native  60238  60305  ...   98536   81239
Asian/Pacific Islander             63226  61033  ...  130416  163228
Black or African American          48915  51082  ...  150416  275000
Hispanic/Latino                    46503  54782  ...  126115  165216
Others                             63785  38771  ...   63785   38771
White                              66793  63940  ...  178331  210588

[6 rows x 6 columns]

## Replicating pivot_table with a groupby aggregation

At first glance, it may seem that the .pivot_table method provides a unique way to
analyze data. However, after a little massaging, it is possible to replicate its functionality
with the .groupby method. Knowing this equivalence can help shrink the universe of pandas
functionality.
In this recipe, we use the flights dataset to create a pivot table and then recreate it using the
.groupby method.

In [90]:
# Read in the flights dataset, and use the .pivot_table 
# method to find the total number of canceled flights
# per origin airport for each airline:
flights = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/flights.csv')

In [91]:
fpt = flights.pivot_table(index='AIRLINE',
                         columns='ORG_AIR',
                         values='CANCELLED',
                         aggfunc='sum',
                         fill_value=0)

In [92]:
fpt

ORG_AIR,ATL,DEN,...,PHX,SFO
AIRLINE,,,,,
AA,3,4,...,4,2
AS,0,0,...,0,0
B6,0,0,...,0,1
DL,28,1,...,1,2
EV,18,6,...,0,0
F9,0,2,...,0,0
HA,0,0,...,0,0
MQ,5,0,...,0,0
NK,1,1,...,2,0


To replicate this with the .groupby method, we will need to groupby two columns
and then unstack them. A groupby aggregation cannot replicate this table. The trick
is to group by all the columns in both the index and columns parameters first:

In [95]:
(flights
 .groupby(['AIRLINE', 'ORG_AIR'])
 ['CANCELLED']
 .sum()
)

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [96]:
# Use the .unstack method to pivot the ORG_AIR index
# level to column names
fpg = (flights
       .groupby(['AIRLINE', 'ORG_AIR'])
       ['CANCELLED']
       .sum()
       .unstack('ORG_AIR', fill_value=0)
)

In [97]:
fpg

ORG_AIR,ATL,DEN,...,PHX,SFO
AIRLINE,,,,,
AA,3,4,...,4,2
AS,0,0,...,0,0
B6,0,0,...,0,1
DL,28,1,...,1,2
EV,18,6,...,0,0
F9,0,2,...,0,0
HA,0,0,...,0,0
MQ,5,0,...,0,0
NK,1,1,...,2,0


In [98]:
fpt.equals(fpg)

True

In [99]:
# It is possible to replicate much more complex pivot 
# tables with the .groupby method. For instance, take 
# the following result from .pivot_table:

flights.pivot_table(index=['AIRLINE', 'MONTH'],
                   columns=['ORG_AIR', 'CANCELLED'],
                   values=['DEP_DELAY', 'DIST'],
                   aggfunc=['sum', 'mean'],
                   fill_value=0)

sum     ...         mean        
              DEP_DELAY     ...         DIST        
ORG_AIR             ATL     ...          SFO        
CANCELLED             0  1  ...            0       1
AIRLINE MONTH               ...                     
AA      1           -13  0  ...  1860.166667     0.0
        2           -39  0  ...  1337.916667  2586.0
        3            -2  0  ...  1502.758621     0.0
        4             1  0  ...  1646.903226     0.0
        5            52  0  ...  1436.892857     0.0
...                 ... ..  ...          ...     ...
WN      7          2604  0  ...   636.210526     0.0
        8          1718  0  ...   644.857143   392.0
        9          1033  0  ...   731.578947   354.5
        11          700  0  ...   580.875000   392.0
        12         1679  0  ...   782.256410     0.0

[149 rows x 80 columns]

To replicate this with the .groupby method, follow the same pattern from the recipe, place all
the columns from the index and columns parameters into the .groupby method, and then call
.unstack to pull the index levels out to the columns:

In [107]:
(flights
 .groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])
 [['DEP_DELAY', 'DIST']]
 .agg(['sum', 'mean'])
 .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0)
 .swaplevel(0, 1, axis=1)
)

sum       ...         mean        
              DEP_DELAY       ...         DIST        
ORG_AIR             ATL       ...          SFO        
CANCELLED             0    1  ...            0       1
AIRLINE MONTH                 ...                     
AA      1         -13.0  0.0  ...  1860.166667     0.0
        2         -39.0  0.0  ...  1337.916667  2586.0
        3          -2.0  0.0  ...  1502.758621     0.0
        4           1.0  0.0  ...  1646.903226     0.0
        5          52.0  0.0  ...  1436.892857     0.0
...                 ...  ...  ...          ...     ...
WN      7        2604.0  0.0  ...   636.210526     0.0
        8        1718.0  0.0  ...   644.857143   392.0
        9        1033.0  0.0  ...   731.578947   354.5
        11        700.0  0.0  ...   580.875000   392.0
        12       1679.0  0.0  ...   782.256410     0.0

[149 rows x 80 columns]

## Renaming axis levels for easy reshaping 

Reshaping with .stack and .unstack methods is far easier when each axis (both index and column) level has a name. pandas allows users to reference each axis level by
integer location or by name. Since integer location is implicit and not explicit, you should
consider using level names whenever possible. This advice follows from The Zen of Python
(type import this if you are not familiar with it), a short list of guiding principles for Python,
of which the second one is "Explicit is better than implicit."
When grouping or aggregating with multiple columns, the resulting pandas object will have
multiple levels in one or both of the axes. In this recipe, we will name each level of each axis
and then use the .stack and .unstack methods to reshape the data to the desired form.

In [112]:
# Read in the college dataset, and find a few basic
# summary statistics on the undergraduate population
# and SAT math scores by institution and religious
# affiliation:
pd.set_option('max_columns', None)
college = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/college.csv', )

In [113]:
(college
 .groupby(['STABBR', 'RELAFFIL'])
 [["UGDS", "SATMTMID"]]
 .agg(['size', 'min', 'max'])
)

UGDS                 SATMTMID              
                size    min      max     size    min    max
STABBR RELAFFIL                                            
AK     0           7  109.0  12865.0        7    NaN    NaN
       1           3   27.0    275.0        3  503.0  503.0
AL     0          72   12.0  29851.0       72  420.0  590.0
       1          24   13.0   3033.0       24  400.0  560.0
AR     0          68   18.0  21405.0       68  427.0  565.0
...              ...    ...      ...      ...    ...    ...
WI     0          87   20.0  29302.0       87  480.0  680.0
       1          25    4.0   8212.0       25  452.0  605.0
WV     0          65   20.0  44924.0       65  430.0  530.0
       1           8   63.0   1375.0        8  455.0  510.0
WY     0          11   52.0   9910.0       11  540.0  540.0

[112 rows x 6 columns]

In [118]:
# Notice that both index levels have names and are the
# old column names. The column levels, on the other hand,
# do not have names. Use the .rename_axis method to
# give them level names:
(college
 .groupby(['STABBR', 'RELAFFIL'])
 [["UGDS", "SATMTMID"]]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns') 
)

AGG_COLS        UGDS                 SATMTMID              
AGG_FUNCS       size    min      max     size    min    max
STABBR RELAFFIL                                            
AK     0           7  109.0  12865.0        7    NaN    NaN
       1           3   27.0    275.0        3  503.0  503.0
AL     0          72   12.0  29851.0       72  420.0  590.0
       1          24   13.0   3033.0       24  400.0  560.0
AR     0          68   18.0  21405.0       68  427.0  565.0
...              ...    ...      ...      ...    ...    ...
WI     0          87   20.0  29302.0       87  480.0  680.0
       1          25    4.0   8212.0       25  452.0  605.0
WV     0          65   20.0  44924.0       65  430.0  530.0
       1           8   63.0   1375.0        8  455.0  510.0
WY     0          11   52.0   9910.0       11  540.0  540.0

[112 rows x 6 columns]

In [124]:
# Now that each axis level has a name, reshaping is a
# breeze. Use the .stack method to move the AGG_FUNCS 
# column to an index level:

(college
 .groupby(['STABBR', 'RELAFFIL'])
 [['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis=1)
 .stack('AGG_FUNCS')
 
)

AGG_COLS                      UGDS  SATMTMID
RELAFFIL STABBR AGG_FUNCS                   
0        AK     size           7.0       7.0
                min          109.0       NaN
                max        12865.0       NaN
1        AK     size           3.0       3.0
                min           27.0     503.0
...                            ...       ...
         WV     min           63.0     455.0
                max         1375.0     510.0
0        WY     size          11.0      11.0
                min           52.0     540.0
                max         9910.0     540.0

[332 rows x 2 columns]

In [133]:
# We can continue to make use of the axis level names by
# sorting levels with the .sort_index method:
(college
 .groupby(['STABBR', 'RELAFFIL'])
 [['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
 .stack('AGG_FUNCS')
 .sort_index(level='RELAFFIL', axis='index')
 .sort_index(level='AGG_COLS', axis='columns')
)

AGG_COLS                   SATMTMID     UGDS
STABBR RELAFFIL AGG_FUNCS                   
AK     0        max             NaN  12865.0
                min             NaN    109.0
                size            7.0      7.0
AL     0        max           590.0  29851.0
                min           420.0     12.0
...                             ...      ...
WI     1        min           452.0      4.0
                size           25.0     25.0
WV     1        max           510.0   1375.0
                min           455.0     63.0
                size            8.0      8.0

[332 rows x 2 columns]

In [137]:
# To completely reshape your data, you might need to
# stack some columns while unstacking others. 
# Chain the two methods together:

(college
 .groupby(['STABBR', 'RELAFFIL'])
 [['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
 .stack('AGG_FUNCS')
  .unstack(['RELAFFIL', 'STABBR'])
)

AGG_COLS      UGDS                                                             \
RELAFFIL         0      1        0       1        0       1       0             
STABBR          AK     AK       AL      AL       AR      AR      AS        AZ   
AGG_FUNCS                                                                       
size           7.0    3.0     72.0    24.0     68.0    18.0     1.0     124.0   
min          109.0   27.0     12.0    13.0     18.0    20.0  1276.0       1.0   
max        12865.0  275.0  29851.0  3033.0  21405.0  4485.0  1276.0  151558.0   

AGG_COLS                                                                       \
RELAFFIL        1        0       1        0       1        0       1        0   
STABBR         AZ       CA      CA       CO      CO       CT      CT       DC   
AGG_FUNCS                                                                       
size          9.0    609.0   164.0    118.0     7.0     85.0    17.0     17.0   
min          25.0      0.0     8.0      0.0     0.0     12.0     0.0     22.0   
max        4102.0  44744.0  6745.0  25873.0  4914.0  18016.0  4904.0  10433.0   

AGG_COLS                                                               \
RELAFFIL        1        0       1        0        1       0            
STABBR         DC       DE      DE       FL       FL      FM       GA   
AGG_FUNCS                                                               
size          9.0     16.0     3.0    347.0     89.0     1.0    147.0   
min        1665.0     42.0  1431.0      5.0     14.0  2344.0     19.0   
max        7211.0  18222.0  6081.0  61470.0  11976.0  2344.0  26738.0   

AGG_COLS                                                                     \
RELAFFIL         1       0     1        0       1        0       1        0   
STABBR          GA      GU    GU       HI      HI       IA      IA       ID   
AGG_FUNCS                                                                     
size          37.0     2.0   1.0     22.0     4.0     62.0    30.0     36.0   
min           88.0  2010.0  65.0      3.0    96.0      9.0     1.0     12.0   
max        23058.0  3607.0  65.0  13667.0  2522.0  40335.0  2675.0  16206.0   

AGG_COLS                                                                 \
RELAFFIL         1        0        1        0        1        0       1   
STABBR          ID       IL       IL       IN       IN       KS      KS   
AGG_FUNCS                                                                 
size           4.0    243.0     57.0     99.0     62.0     68.0    31.0   
min           41.0      4.0      6.0      4.0     77.0     32.0    17.0   
max        23865.0  31875.0  15858.0  77657.0  10218.0  19727.0  1836.0   

AGG_COLS                                                                       \
RELAFFIL         0       1        0       1        0       1        0       1   
STABBR          KY      KY       LA      LA       MA      MA       MD      MD   
AGG_FUNCS                                                                       
size          81.0    25.0    109.0    12.0    163.0    31.0     84.0    15.0   
min           18.0    24.0     15.0    12.0      2.0    27.0      7.0    17.0   
max        21725.0  2499.0  24452.0  2734.0  21864.0  9483.0  33892.0  4062.0   

AGG_COLS                                                                      \
RELAFFIL        0       1       0                1        0       1        0   
STABBR         ME      ME      MH       MI      MI       MN      MN       MO   
AGG_FUNCS                                                                      
size         37.0     6.0     1.0    159.0    48.0    113.0    41.0    154.0   
min          41.0   334.0  1078.0     12.0   160.0      1.0    43.0     11.0   
max        8682.0  2292.0  1078.0  38395.0  3905.0  30135.0  6080.0  27276.0   

AGG_COLS                                                                      \
RELAFFIL        1       0                1        0       1    

In [138]:
# Stack all the columns at once to return a Series:

(college
 .groupby(['STABBR', 'RELAFFIL'])
 [['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
 .stack(['AGG_FUNCS', 'AGG_COLS'])
)

STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         size       UGDS            7.0
                             SATMTMID        7.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         size       UGDS            3.0
                                          ...   
WY      0         size       SATMTMID       11.0
                  min        UGDS           52.0
                             SATMTMID      540.0
                  max        UGDS         9910.0
                             SATMTMID      540.0
Length: 640, dtype: float64

In [141]:
# We can also unstack everything in the index. In this
# case, it collapses to a very wide result, which
# pandas displays as a Series:

(college
 .groupby(['STABBR', 'RELAFFIL'])
 [['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
 .unstack(['STABBR', 'RELAFFIL'])
)

AGG_COLS  AGG_FUNCS  STABBR  RELAFFIL
UGDS      size       AK      0             7.0
                             1             3.0
                     AL      0            72.0
                             1            24.0
                     AR      0            68.0
                                         ...  
SATMTMID  max        WI      1           605.0
                     WV      0           530.0
                             1           510.0
                     WY      0           540.0
                             1             NaN
Length: 708, dtype: float64

If you wish to dispose of the level values altogether, you may set them to None. You can do
this when you want to reduce visual clutter or when it is obvious what the column levels
represent and no further processing will take place:

In [153]:
(college
 .groupby(['STABBR', 'RELAFFIL'])
 [['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis([None, None], axis='index')
 .rename_axis([None, None], axis='columns')
)

UGDS                 SATMTMID              
     size    min      max     size    min    max
AK 0    7  109.0  12865.0        7    NaN    NaN
   1    3   27.0    275.0        3  503.0  503.0
AL 0   72   12.0  29851.0       72  420.0  590.0
   1   24   13.0   3033.0       24  400.0  560.0
AR 0   68   18.0  21405.0       68  427.0  565.0
...   ...    ...      ...      ...    ...    ...
WI 0   87   20.0  29302.0       87  480.0  680.0
   1   25    4.0   8212.0       25  452.0  605.0
WV 0   65   20.0  44924.0       65  430.0  530.0
   1    8   63.0   1375.0        8  455.0  510.0
WY 0   11   52.0   9910.0       11  540.0  540.0

[112 rows x 6 columns]

### Tidying when multiple variable are stored as column names

One particular flavor of messy data appears whenever the column names contain multiple
different variables themselves. A common example of this scenario occurs when age and sex
are concatenated together. To tidy datasets like this, we must manipulate the columns with
the pandas .str attribute. This attribute contains additional methods for string processing.
In this recipe, we will first identify all the variables, of which some will be concatenated
together as column names. We then reshape the data and parse the text to extract the
correct variable values.

In [157]:
# Read in the men's weightlifting dataset, and identify
# the variables:
pd.set_option('max_columns', 5, 'max_rows', 15)

In [158]:

weightlifting = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/weightlifting_men.csv')

In [183]:
weightlifting

,Weight Category,M35 35-39,...,M75 75-79,M80 80+
0,56,137,...,62,55
1,62,152,...,67,57
2,69,167,...,75,60
3,77,182,...,82,65
4,85,192,...,87,70
5,94,202,...,90,75
6,105,210,...,95,80
7,105+,217,...,100,85


The variables are the Weight Category, a combination of sex and age, and the
qualifying total. The age and sex variables have been concatenated together into a
single cell. Before we can separate them, let's use the .melt method to transpose
the age and sex column names into a single vertical column

In [160]:
(weightlifting
 .melt(id_vars='Weight Category',
       
      var_name='sex_age',
      value_name='Qual Total')
)

,Weight Category,sex_age,Qual Total
0,56,M35 35-39,137
1,62,M35 35-39,152
2,69,M35 35-39,167
3,77,M35 35-39,182
4,85,M35 35-39,192
...,...,...,...
75,77,M80 80+,65
76,85,M80 80+,70
77,94,M80 80+,75
78,105,M80 80+,80


In [164]:
# Select the sex_age column, and use the .split method
# available from the .str attribute to split the column
# into two different columns:
(weightlifting
 .melt(id_vars='Weight Category',
       
      var_name='sex_age',
      value_name='Qual Total')
 ['sex_age']
 .str.split(expand=True)
)

,0,1
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39
...,...,...
75,M80,80+
76,M80,80+
77,M80,80+
78,M80,80+


In [162]:
# This operation returned a DataFrame with meaningless 
# column names. Let's rename the columns:

(weightlifting
 .melt(id_vars='Weight Category',
       
      var_name='sex_age',
      value_name='Qual Total')
 ['sex_age']
 .str.split(expand=True)
 .rename(columns={0:'Sex', 1:'Age Group'})
)

,Sex,Age Group
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39
...,...,...
75,M80,80+
76,M80,80+
77,M80,80+
78,M80,80+


In [167]:
# Create a Sex column using an index operation after
# the .str attribute to select the first character 
# from the renamed Sex column:

(weightlifting
 .melt(id_vars='Weight Category',
       
      var_name='sex_age',
      value_name='Qual Total')
 ['sex_age']
 .str.split(expand=True)
 .rename(columns={0:'Sex', 1:'Age Group'})
 .assign(Sex=lambda df_: df_.Sex.str[0])
)

,Sex,Age Group
0,M,35-39
1,M,35-39
2,M,35-39
3,M,35-39
4,M,35-39
...,...,...
75,M,80+
76,M,80+
77,M,80+
78,M,80+


In [168]:
# Use the pd.concat function to concatenate this 
# DataFrame with the Weight Category and 
# Qual Total columns:
melted = (weightlifting
 .melt(id_vars='Weight Category',
       
      var_name='sex_age',
      value_name='Qual Total')
)


In [177]:
tidy = pd.concat([melted
                  ['sex_age']
                 .str.split(expand=True)
                  .rename(columns={0:'Sex', 1:'Age Group'})
                  .assign(Sex=lambda df_: df_.Sex.str[0]),
                  melted[['Weight Category', 'Qual Total']]],
                 axis='columns'
                )

In [178]:
tidy

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192
...,...,...,...,...
75,M,80+,77,65
76,M,80+,85,70
77,M,80+,94,75
78,M,80+,105,80


In [179]:
# This same result could have been created with the
# following:
melted = (weightlifting
 .melt(id_vars='Weight Category',
       
      var_name='sex_age',
      value_name='Qual Total')
)


In [181]:
(melted['sex_age']
 .str.split(expand=True)
 .rename(columns={0:'Sex', 1:'Age Group'})
 .assign(Sex=lambda df_: df_.Sex.str[0],
        Category=melted['Weight Category'],
        Total=melted['Qual Total'])
 .rename(columns={'Category': 'Weight Category', 'Total': 'Qual Total'})
)

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192
...,...,...,...,...
75,M,80+,77,65
76,M,80+,85,70
77,M,80+,94,75
78,M,80+,105,80


In [184]:
tidy2 = (weightlifting
         .melt(id_vars='Weight Category',
           var_name='sex_age',
           value_name='Qual Total')
         .assign(Sex=lambda df_:df_.sex_age.str[0],
         **{'Age Group':(lambda df_: (df_
          .sex_age
          .str.extract(r'(\d{2}[-+](?:\d{2})?)',
                       expand=False)))})
               .drop(columns='sex_age')
)

In [185]:
tidy2

,Weight Category,Qual Total,Sex,Age Group
0,56,137,M,35-39
1,62,152,M,35-39
2,69,167,M,35-39
3,77,182,M,35-39
4,85,192,M,35-39
...,...,...,...,...
75,77,65,M,80+
76,85,70,M,80+
77,94,75,M,80+
78,105,80,M,80+


### Tidying when multiple variables are stored as a single column

Tidy datasets must have a single column for each variable. Occasionally, multiple variable
names are placed in a single column with their corresponding value placed in another.
In this recipe, we identify the column containing the improperly structured variables and pivot
it to create tidy data.

In [188]:
# Read in the restaurant inspections dataset, and convert
# the Date column data type to datetime64:
inspections = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/restaurant_inspections.csv', parse_dates=['Date'])

In [189]:
inspections

,Name,Date,Info,Value
0,E & E Grill House,2017-08-08,Borough,MANHATTAN
1,E & E Grill House,2017-08-08,Cuisine,American
2,E & E Grill House,2017-08-08,Description,Non-food contact surface improperly constructe...
3,E & E Grill House,2017-08-08,Grade,A
4,E & E Grill House,2017-08-08,Score,9.0
...,...,...,...,...
495,PIER SIXTY ONE-THE LIGHTHOUSE,2017-09-01,Borough,MANHATTAN
496,PIER SIXTY ONE-THE LIGHTHOUSE,2017-09-01,Cuisine,American
497,PIER SIXTY ONE-THE LIGHTHOUSE,2017-09-01,Description,Filth flies or food/refuse/sewage-associated (...
498,PIER SIXTY ONE-THE LIGHTHOUSE,2017-09-01,Grade,Z


This dataset has two columns, Name and Date, that are each correctly contained
in a single column. The Info column has five different variables: Borough, Cuisine,
Description, Grade, and Score. Let's attempt to use the .pivot method to keep the
Name and Date columns vertical, create new columns out of all the values in the
Info column, and use the Value column as their intersection:

In [198]:
inspections.pivot(index=['Name', 'Date'],
                 columns='Info', values='Value')

,Info,Borough,Cuisine,Description,Grade,Score
Name,Date,,,,,
3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...
VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0


In [201]:
(inspections
.groupby(['Name', 'Date'])
[['Info']])

Unfortunately, pandas developers have not implemented this functionality for us.
Thankfully, for the most part, pandas has multiple ways of accomplishing the same
task. Let's put Name, Date, and Info into the index:

In [199]:
inspections.set_index(['Name', 'Date', 'Info'])

Value
Name                          Date       Info                                                          
E & E Grill House             2017-08-08 Borough                                              MANHATTAN
                                         Cuisine                                               American
                                         Description  Non-food contact surface improperly constructe...
                                         Grade                                                        A
                                         Score                                                      9.0
...                                                                                                 ...
PIER SIXTY ONE-THE LIGHTHOUSE 2017-09-01 Borough                                              MANHATTAN
                                         Cuisine                                               American
                                         Description  Filth flies or food/refuse/sewage-associated (...
                                         Grade                                                        Z
                                         Score                                                     33.0

[500 rows x 1 columns]

In [202]:
# Use the .unstack method to pivot all the values in
# the Info column:
(
    inspections
    .set_index(['Name', 'Date', 'Info'])
    .unstack('Info')
)

Value  \
Info                                                  Borough   
Name                                Date                        
3 STAR JUICE CENTER                 2017-05-10       BROOKLYN   
A & L PIZZA RESTAURANT              2017-08-22       BROOKLYN   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25       BROOKLYN   
ANTOJITOS DELI FOOD                 2017-06-01       BROOKLYN   
BANGIA                              2017-06-16      MANHATTAN   
...                                                       ...   
VALL'S PIZZERIA                     2017-03-15  STATEN ISLAND   
VIP GRILL                           2017-06-12       BROOKLYN   
WAHIZZA                             2017-04-13      MANHATTAN   
WANG MANDOO HOUSE                   2017-08-29         QUEENS   
XIAOYAN YABO INC                    2017-08-29         QUEENS   

                                                                                                   \
Info                                                                                      Cuisine   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10                     Juice, Smoothies, Fruit Salads   
A & L PIZZA RESTAURANT              2017-08-22                                              Pizza   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25                                            Turkish   
ANTOJITOS DELI FOOD                 2017-06-01  Latin (Cuban, Dominican, Puerto Rican, South &...   
BANGIA                              2017-06-16                                             Korean   
...                                                                                           ...   
VALL'S PIZZERIA                     2017-03-15                                      Pizza/Italian   
VIP GRILL                           2017-06-12                                      Jewish/Kosher   
WAHIZZA                             2017-04-13                                              Pizza   
WANG MANDOO HOUSE                   2017-08-29                                             Korean   
XIAOYAN YABO INC                    2017-08-29                                             Korean   

                                                                                                   \
Info                                                                                  Description   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10  Facility not vermin proof. Harborage or condit...   
A & L PIZZA RESTAURANT              2017-08-22  Facility not vermin proof. Harborage or condit...   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25  Plumbing not properly installed or maintained;...   
ANTOJITOS DELI FOOD                 2017-06-01  Live roaches present in facility's food and/or...   
BANGIA                              2017-06-16  Covered garbage receptacle not provided or ina...   
...                                                                                           ...   
VALL'S PIZZERIA                     2017-03-15  Wiping cloths soiled or not stored in sanitizi...   
VIP GRILL                           2017-06-12        Hot food item not held at or above 140Âº F.   
WAHIZZA                             2017-04-13  No facilities available to wash, rinse and san...   
WANG MANDOO HOUSE                   2017-08-29  Accurate thermometer not provided in refrigera...   
XIAOYAN YABO INC                    2017-08-29  Evidence of mice or live mice present in facil...   

                                                            
Info                                           Grade Score  
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10     A  12.0  
A & L PIZZA RESTAURANT              2017-08-22     A   9.0  
AKSARAY TU

In [207]:
# Make the index levels into columns with the
# .reset_index method:
(inspections
    .set_index(['Name', 'Date', 'Info'])
 .unstack('Info')
 .reset_index(col_level=-1)
)

... Value      
Info                                 Name       Date  ... Grade Score
0                     3 STAR JUICE CENTER 2017-05-10  ...     A  12.0
1                  A & L PIZZA RESTAURANT 2017-08-22  ...     A   9.0
2     AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25  ...     A  13.0
3                     ANTOJITOS DELI FOOD 2017-06-01  ...     A  10.0
4                                  BANGIA 2017-06-16  ...     A   9.0
..                                    ...        ...  ...   ...   ...
95                        VALL'S PIZZERIA 2017-03-15  ...     A   9.0
96                              VIP GRILL 2017-06-12  ...     A  10.0
97                                WAHIZZA 2017-04-13  ...     A  10.0
98                      WANG MANDOO HOUSE 2017-08-29  ...     A  12.0
99                       XIAOYAN YABO INC 2017-08-29  ...     Z  49.0

[100 rows x 7 columns]

The dataset is tidy, but there is some annoying leftover pandas debris that needs to
be removed. Let's use the .droplevel method to remove the top column level and
then rename the index level to None:

In [210]:
(
    inspections
    .set_index(['Name', 'Date', 'Info'])
    .unstack('Info')
    .reset_index(col_level=-1)
    .droplevel(0, axis=1)
    .rename_axis(None, axis=1)
)

,Name,Date,...,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,...,A,10.0
4,BANGIA,2017-06-16,...,A,9.0
...,...,...,...,...,...
95,VALL'S PIZZERIA,2017-03-15,...,A,9.0
96,VIP GRILL,2017-06-12,...,A,10.0
97,WAHIZZA,2017-04-13,...,A,10.0
98,WANG MANDOO HOUSE,2017-08-29,...,A,12.0


The creation of the column MultiIndex in step 4 could have been avoided by
converting that one column DataFrame in step 3 into a Series with the .squeeze
method. The following code produces the same result as the previous step:

In [211]:
(
    inspections
    .set_index(['Name', 'Date', 'Info'])
    .squeeze()
    .unstack('Info')
    .reset_index()
    .rename_axis(None, axis=1)
)

,Name,Date,...,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,...,A,10.0
4,BANGIA,2017-06-16,...,A,9.0
...,...,...,...,...,...
95,VALL'S PIZZERIA,2017-03-15,...,A,9.0
96,VIP GRILL,2017-06-12,...,A,10.0
97,WAHIZZA,2017-04-13,...,A,10.0
98,WANG MANDOO HOUSE,2017-08-29,...,A,12.0


It is possible to use the .pivot_table method, which has no restrictions on how many nonpivoted
columns are allowed. The .pivot_table method differs from .pivot by performing
an aggregation for all the values that correspond to the intersection between the columns in
the index and columns parameters. Because there may be multiple values in this intersection, .pivot_table requires the user
to pass it an aggregating function to output a single value. We use the first aggregating
function, which takes the first of the values of the group. In this particular example, there
is exactly one value for each intersection, so there is nothing to be aggregated. The default
aggregation function is the mean, which will produce an error here, since some of the values
are strings:

In [217]:
(inspections.pivot_table(index=['Name', 'Date'],
                 columns='Info', values='Value',
                       aggfunc='first')
.reset_index()
.rename_axis(None, axis=1)
)

,Name,Date,...,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,...,A,10.0
4,BANGIA,2017-06-16,...,A,9.0
...,...,...,...,...,...
95,VALL'S PIZZERIA,2017-03-15,...,A,9.0
96,VIP GRILL,2017-06-12,...,A,10.0
97,WAHIZZA,2017-04-13,...,A,10.0
98,WANG MANDOO HOUSE,2017-08-29,...,A,12.0


## Tidying when two or more values are stored in the same cell

Tabular data, by nature, is two-dimensional, and thus, there is a limited amount of information
that can be presented in a single cell. As a workaround, you will occasionally see datasets
with more than a single value stored in the same cell. Tidy data allows for just a single value
for each cell. To rectify these situations, you will typically need to parse the string data into
multiple columns with the methods from the .str attribute.
In this recipe, we examine a dataset that has a column containing multiple different variables
in each cell. We use the .str attribute to parse these strings into separate columns to tidy
the data.

In [3]:
cities = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/texas_cities.csv')

In [4]:
cities

,City,Geolocation
0,Houston,"29.7604° N, 95.3698° W"
1,Dallas,"32.7767° N, 96.7970° W"
2,Austin,"30.2672° N, 97.7431° W"


The City column looks good and contains exactly one value. The Geolocation
column, on the other hand, contains four variables: latitude, latitude direction,
longitude, and longitude direction. Let's split the Geolocation column into four
separate columns. We will use the regular expression that matches any character
followed by a space:

In [5]:
geolocations = cities.Geolocation.str.split(pat='. ',
                                           expand=True)

In [6]:
geolocations

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [8]:
geolocations.columns = [ 'latitude', 'latitude direction',
                       'longitude', 'longitude direction']

In [9]:
# Because the original data type for the Geolocation
# was an object, all the new columns are also objects.
# Let's change latitude and longitude into float types:

geolocations = geolocations.astype({'latitude': 'float', 'longitude':'float'})

In [12]:
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [13]:
geolocations

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [14]:
# Combine these new columns with the City column from
# the original:
(geolocations
 .assign(city=cities['City'])
)

,latitude,latitude direction,...,longitude direction,city
0,29.7604,N,...,W,Houston
1,32.7767,N,...,W,Dallas
2,30.2672,N,...,W,Austin


Instead of using a dictionary, which would require a lot of typing if you had many column
names, you can use the function to_numeric to attempt to convert each column to either
integer or float. To apply this function iteratively over each column, use the .apply
method with the following:

In [16]:
geolocations.apply(pd.to_numeric, errors='ignore')

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [20]:
(geolocations.apply(pd.to_numeric, errors='ignore')
.assign(city=cities['City']))

,latitude,latitude direction,...,longitude direction,city
0,29.7604,N,...,W,Houston
1,32.7767,N,...,W,Dallas
2,30.2672,N,...,W,Austin


The .split method worked well in this example with a regular expression. For other
examples, some columns might require you to create splits on several different patterns.
To search for multiple regular expressions, use the pipe character (|). For instance, if we
wanted to split only the degree symbol and comma, each followed by a space, we would
do the following:

In [21]:
cities.Geolocation.str.split(pat=r'° |, ', expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


The .extract method is another method that allows you to extract specific groups within
each cell. These capture groups must be enclosed in parentheses. Anything that matches
outside the parentheses is not present in the result. The following line produces the same
output as step 2:

In [24]:
# {.sourceCode .pycon}
(cities.Geolocation.str.extract(r'([0-9.]+). (N|S), ([0-9]+). (E|W)',
 expand=True)
)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN


## Tidying when variables are stored in column names and values

One particularly difficult form of messy data to diagnose appears whenever variables are
stored both horizontally across the column names and vertically down column values.
This type of dataset usually is not found in a database, but from a summarized report that
someone else has already generated.

In [28]:
pd.set_option('max_columns', None)
sensors = pd.read_csv('C:/Users/justine.o_kobo360/Desktop/Pandas Workbook/Pandas CookBook 1.x/Data files/sensors.csv')

In [29]:
sensors

,Group,Property,2012,2013,2014,2015,2016
0,A,Pressure,928,873,814,973,870
1,A,Temperature,1026,1038,1009,1036,1042
2,A,Flow,819,806,861,882,856
3,B,Pressure,817,877,914,806,942
4,B,Temperature,1008,1041,1009,1002,1013
5,B,Flow,887,899,837,824,873


The only variable placed correctly in a vertical column is Group. The Property
column appears to have three unique variables, Pressure, Temperature, and
Flow. The rest of the columns 2012 to 2016 are themselves a single variable, which
we can sensibly name Year. It isn't possible to restructure this kind of messy data
with a single DataFrame method. Let's begin with the .melt method to pivot the
years into their own column:

In [30]:
sensors.melt(id_vars=['Group', 
                     'Property'], var_name='Year')

,Group,Property,Year,value
0,A,Pressure,2012,928
1,A,Temperature,2012,1026
2,A,Flow,2012,819
3,B,Pressure,2012,817
4,B,Temperature,2012,1008
...,...,...,...,...
25,A,Temperature,2016,1042
26,A,Flow,2016,856
27,B,Pressure,2016,942
28,B,Temperature,2016,1013


In [31]:
# This takes care of one of our issues. Let's use the
# .pivot_table method to pivot
# the Property column into new column names:

(sensors
  .melt(id_vars=['Group', 'Property'], var_name='Year')
  .pivot_table(index=['Group', 'Year'],
    columns='Property', values='value')
    .reset_index()
     .rename_axis(None, axis='columns')
)

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


Whenever a solution involves .melt, .pivot_table, or .pivot, you can be sure that there
is an alternative method using .stack and .unstack. The trick is first to move the columns
that are not currently being pivoted into the index:

In [38]:
(sensors
    .set_index(['Group', 'Property'])
    .rename_axis('Year', axis='columns')
     .stack()
     .unstack('Property')
    .rename_axis(None, axis='columns')
    .reset_index()
)

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013
